In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

# Define the EXP3 algorithm class
class EXP3:
    def __init__(self, n_arms, gamma=0.1):
        self.n_arms = n_arms
        self.gamma = gamma
        self.weights = np.ones(n_arms)
        
    def select_arm(self):
        total_weight = np.sum(self.weights)
        probabilities = (
            (1 - self.gamma) * (self.weights / total_weight) +
            (self.gamma / self.n_arms)
        )
        arm = np.random.choice(self.n_arms, p=probabilities)
        return arm, probabilities
        
    def update(self, chosen_arm, reward, probabilities):
        x = reward / probabilities[chosen_arm]
        self.weights[chosen_arm] *= np.exp((self.gamma * x) / self.n_arms)

# Load the House Prices dataset
data = pd.read_csv('train2.csv')

# Preprocessing steps
# Drop columns with too many missing values or irrelevant features
data = data.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu'], axis=1)

# Separate features and target
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

# Handle missing values
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(include=[object]).columns

# Impute numerical features with median
imputer_num = SimpleImputer(strategy='median')
X[numeric_features] = imputer_num.fit_transform(X[numeric_features])

# Impute categorical features with mode
imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_features] = imputer_cat.fit_transform(X[categorical_features])

# Encode categorical variables
label_encoders = {}
for column in categorical_features:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

# Split the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the possible values for each hyperparameter with expanded search space
hyperparameter_values = {
    'n_estimators': list(range(50, 301, 10)),  # [50, 60, ..., 300]
    'max_depth': [None] + list(range(5, 36, 5)),  # [None, 5, 10, ..., 35]
    'min_samples_split': list(range(2, 21, 2)),  # [2, 4, ..., 20]
    'max_features': ['sqrt', 'log2', None]
}

# Initialize current hyperparameter values (starting point)
current_params = {
    'n_estimators': 50,
    'max_depth': 5,
    'min_samples_split': 2,
    'max_features': None
}

# **New Section: Train and Evaluate the Initial Model**
# Train the Random Forest model with the initial hyperparameters
initial_model = RandomForestRegressor(
    n_estimators=current_params['n_estimators'],
    max_depth=current_params['max_depth'],
    min_samples_split=current_params['min_samples_split'],
    max_features=current_params['max_features'],
    random_state=42
)
initial_model.fit(X_train, y_train)

# Evaluate the initial model on the validation set
initial_y_pred = initial_model.predict(X_valid)
initial_r2 = r2_score(y_valid, initial_y_pred)
print(f"Initial Model R² Score (Before Tuning): {initial_r2:.4f}")

# Number of iterations for the hyperparameter tuning loop
n_iterations = 20

# Initialize the bandit algorithms for each hyperparameter
bandits = {}
for hp in hyperparameter_values:
    n_arms = len(hyperparameter_values[hp])
    bandits[hp] = {
        'algorithm': EXP3(n_arms)
    }

# Initialize the bandit algorithm for hyperparameter selection
hyperparameters = list(hyperparameter_values.keys())
n_hyperparameters = len(hyperparameters)
hyperparameter_bandit = EXP3(n_hyperparameters)

# Map hyperparameter indices to names
hyperparameter_indices = {idx: hp for idx, hp in enumerate(hyperparameters)}

# Track the best model performance and parameters
best_r2 = initial_r2  # Start with the initial R² score
best_params = current_params.copy()

# Hyperparameter tuning loop
for i in range(n_iterations):
    print(f"\nIteration {i+1}/{n_iterations}")
    
    # Select which hyperparameter to adjust using the hyperparameter bandit
    hp_arm_index, hp_probabilities = hyperparameter_bandit.select_arm()
    hp_to_adjust = hyperparameter_indices[hp_arm_index]
    
    # Select the value for the chosen hyperparameter using its bandit algorithm
    algorithm = bandits[hp_to_adjust]['algorithm']
    arm_index, probabilities = algorithm.select_arm()
    hp_value = hyperparameter_values[hp_to_adjust][arm_index]
    
    # Save the previous value to revert if necessary
    previous_value = current_params[hp_to_adjust]
    
    # Update the current parameters with the new value
    current_params[hp_to_adjust] = hp_value
    
    # Train the Random Forest model with the updated hyperparameters
    model = RandomForestRegressor(
        n_estimators=current_params['n_estimators'],
        max_depth=current_params['max_depth'],
        min_samples_split=current_params['min_samples_split'],
        max_features=current_params['max_features'],
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Evaluate the model on the validation set
    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    reward = r2  # Use R² score as the reward

    # Update the bandit algorithm for the chosen hyperparameter value
    algorithm.update(arm_index, reward, probabilities)
    
    # Update the hyperparameter selection bandit
    hyperparameter_bandit.update(hp_arm_index, reward, hp_probabilities)
    
    # Print detailed output
    print(f"Adjusted Hyperparameter: {hp_to_adjust}")
    print(f"Chosen Value for '{hp_to_adjust}': {hp_value}")
    print(f"Reward (R² Score): {reward:.4f}")
    
    # Check if the new configuration is better
    if r2 > best_r2:
        best_r2 = r2
        best_params = current_params.copy()
        print(f"New Best R² Score: {best_r2:.4f} with Parameters: {best_params}")
    else:
        # If not better, revert the hyperparameter change
        current_params[hp_to_adjust] = previous_value

# After the tuning loop, print the best hyperparameters and R² score
print("\nBest Hyperparameters Found:")
print(best_params)
print(f"Best Validation R² Score: {best_r2:.4f}")

# Train the final model with the best hyperparameters
final_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    max_features=best_params['max_features'],
    random_state=42
)
final_model.fit(X_train, y_train)

# Make predictions on the validation set
final_predictions = final_model.predict(X_valid)

# Compare final predictions with actual values
final_comparison_df = pd.DataFrame({'Actual': y_valid.values, 'Predicted': final_predictions})
print("\nFinal Predictions vs. Actual Values:")
print(final_comparison_df.head(20))

Initial Model R² Score (Before Tuning): 0.8588

Iteration 1/20
Adjusted Hyperparameter: min_samples_split
Chosen Value for 'min_samples_split': 10
Reward (R² Score): 0.8607
New Best R² Score: 0.8607 with Parameters: {'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 10, 'max_features': None}

Iteration 2/20
Adjusted Hyperparameter: n_estimators
Chosen Value for 'n_estimators': 120
Reward (R² Score): 0.8643
New Best R² Score: 0.8643 with Parameters: {'n_estimators': 120, 'max_depth': 5, 'min_samples_split': 10, 'max_features': None}

Iteration 3/20
Adjusted Hyperparameter: max_depth
Chosen Value for 'max_depth': 20
Reward (R² Score): 0.8865
New Best R² Score: 0.8865 with Parameters: {'n_estimators': 120, 'max_depth': 20, 'min_samples_split': 10, 'max_features': None}

Iteration 4/20
Adjusted Hyperparameter: max_features
Chosen Value for 'max_features': None
Reward (R² Score): 0.8865

Iteration 5/20
Adjusted Hyperparameter: n_estimators
Chosen Value for 'n_estimators': 200
Reward 

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

# Define the EXP3 algorithm class
class EXP3:
    def __init__(self, n_arms, gamma=0.1):
        self.n_arms = n_arms
        self.gamma = gamma
        self.weights = np.ones(n_arms)
        
    def select_arm(self):
        total_weight = np.sum(self.weights)
        probabilities = (
            (1 - self.gamma) * (self.weights / total_weight) +
            (self.gamma / self.n_arms)
        )
        arm = np.random.choice(self.n_arms, p=probabilities)
        return arm, probabilities
        
    def update(self, chosen_arm, reward, probabilities):
        x = reward / probabilities[chosen_arm]
        self.weights[chosen_arm] *= np.exp((self.gamma * x) / self.n_arms)

# Load the House Prices dataset
data = pd.read_csv('train2.csv')

# Preprocessing steps
# Drop columns with too many missing values or irrelevant features
data = data.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu'], axis=1)

# Separate features and target
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

# Handle missing values
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(include=[object]).columns

# Impute numerical features with median
imputer_num = SimpleImputer(strategy='median')
X[numeric_features] = imputer_num.fit_transform(X[numeric_features])

# Impute categorical features with mode
imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_features] = imputer_cat.fit_transform(X[categorical_features])

# Encode categorical variables
label_encoders = {}
for column in categorical_features:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

# Split the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the possible values for each hyperparameter with expanded search space
hyperparameter_values = {
    'n_estimators': list(range(50, 301, 10)),  # [50, 60, ..., 300]
    'max_depth': [None] + list(range(5, 36, 5)),  # [None, 5, 10, ..., 35]
    'min_samples_split': list(range(2, 21, 2)),  # [2, 4, ..., 20]
    'max_features': ['sqrt', 'log2', None]
}

# Initialize current hyperparameter values (starting point)
current_params = {
    'n_estimators': 50,
    'max_depth': 5,
    'min_samples_split': 2,
    'max_features': None
}

# **New Section: Train and Evaluate the Initial Model**
# Train the Random Forest model with the initial hyperparameters
initial_model = RandomForestRegressor(
    n_estimators=current_params['n_estimators'],
    max_depth=current_params['max_depth'],
    min_samples_split=current_params['min_samples_split'],
    max_features=current_params['max_features'],
    random_state=42
)
initial_model.fit(X_train, y_train)

# Evaluate the initial model on the validation set
initial_y_pred = initial_model.predict(X_valid)
initial_r2 = r2_score(y_valid, initial_y_pred)
print(f"Initial Model R² Score (Before Tuning): {initial_r2:.4f}")

# Number of iterations for the hyperparameter tuning loop
n_iterations = 50

# Initialize the bandit algorithms for each hyperparameter
bandits = {}
for hp in hyperparameter_values:
    n_arms = len(hyperparameter_values[hp])
    bandits[hp] = {
        'algorithm': EXP3(n_arms)
    }

# Initialize the bandit algorithm for hyperparameter selection
hyperparameters = list(hyperparameter_values.keys())
n_hyperparameters = len(hyperparameters)
hyperparameter_bandit = EXP3(n_hyperparameters)

# Map hyperparameter indices to names
hyperparameter_indices = {idx: hp for idx, hp in enumerate(hyperparameters)}

# Track the best model performance and parameters
best_r2 = initial_r2  # Start with the initial R² score
best_params = current_params.copy()

# Hyperparameter tuning loop
for i in range(n_iterations):
    print(f"\nIteration {i+1}/{n_iterations}")
    
    # Select which hyperparameter to adjust using the hyperparameter bandit
    hp_arm_index, hp_probabilities = hyperparameter_bandit.select_arm()
    hp_to_adjust = hyperparameter_indices[hp_arm_index]
    
    # Select the value for the chosen hyperparameter using its bandit algorithm
    algorithm = bandits[hp_to_adjust]['algorithm']
    arm_index, probabilities = algorithm.select_arm()
    hp_value = hyperparameter_values[hp_to_adjust][arm_index]
    
    # Save the previous value to revert if necessary
    previous_value = current_params[hp_to_adjust]
    
    # Update the current parameters with the new value
    current_params[hp_to_adjust] = hp_value
    
    # Train the Random Forest model with the updated hyperparameters
    model = RandomForestRegressor(
        n_estimators=current_params['n_estimators'],
        max_depth=current_params['max_depth'],
        min_samples_split=current_params['min_samples_split'],
        max_features=current_params['max_features'],
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Evaluate the model on the validation set
    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    reward = r2  # Use R² score as the reward

    # Update the bandit algorithm for the chosen hyperparameter value
    algorithm.update(arm_index, reward, probabilities)
    
    # Update the hyperparameter selection bandit
    hyperparameter_bandit.update(hp_arm_index, reward, hp_probabilities)
    
    # Print detailed output
    print(f"Adjusted Hyperparameter: {hp_to_adjust}")
    print(f"Chosen Value for '{hp_to_adjust}': {hp_value}")
    print(f"Reward (R² Score): {reward:.4f}")
    
    # Check if the new configuration is better
    if r2 > best_r2:
        best_r2 = r2
        best_params = current_params.copy()
        print(f"New Best R² Score: {best_r2:.4f} with Parameters: {best_params}")
    else:
        # If not better, revert the hyperparameter change
        current_params[hp_to_adjust] = previous_value

# After the tuning loop, print the best hyperparameters and R² score
print("\nBest Hyperparameters Found:")
print(best_params)
print(f"Best Validation R² Score: {best_r2:.4f}")

# Train the final model with the best hyperparameters
final_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    max_features=best_params['max_features'],
    random_state=42
)
final_model.fit(X_train, y_train)

# Make predictions on the validation set
final_predictions = final_model.predict(X_valid)

# Compare final predictions with actual values
final_comparison_df = pd.DataFrame({'Actual': y_valid.values, 'Predicted': final_predictions})
print("\nFinal Predictions vs. Actual Values:")
print(final_comparison_df.head(20))

Initial Model R² Score (Before Tuning): 0.8588

Iteration 1/50
Adjusted Hyperparameter: min_samples_split
Chosen Value for 'min_samples_split': 10
Reward (R² Score): 0.8607
New Best R² Score: 0.8607 with Parameters: {'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 10, 'max_features': None}

Iteration 2/50
Adjusted Hyperparameter: n_estimators
Chosen Value for 'n_estimators': 130
Reward (R² Score): 0.8644
New Best R² Score: 0.8644 with Parameters: {'n_estimators': 130, 'max_depth': 5, 'min_samples_split': 10, 'max_features': None}

Iteration 3/50
Adjusted Hyperparameter: min_samples_split
Chosen Value for 'min_samples_split': 6
Reward (R² Score): 0.8617

Iteration 4/50
Adjusted Hyperparameter: n_estimators
Chosen Value for 'n_estimators': 90
Reward (R² Score): 0.8638

Iteration 5/50
Adjusted Hyperparameter: max_depth
Chosen Value for 'max_depth': 20
Reward (R² Score): 0.8864
New Best R² Score: 0.8864 with Parameters: {'n_estimators': 130, 'max_depth': 20, 'min_samples_split': 1